In [1]:
%load_ext rich

# Prompts & modules


In [2]:
import json

from promptimus import Message, MessageRole, Module, Prompt
from promptimus.llms import OllamaProvider

In [3]:
# creating a provider
provider = OllamaProvider(model_name="phi4", base_url="http://lilan:11434/v1")

## Prompts

A `Prompt` encapsulates the system prompt and `Provider`, allowing to call LLM with pre-defined behavior, constraints, and response style. 
Core Functionality

-  Encapsulates the system prompt, enforcing predefined behavior.
-  Requires an LLM provider to execute and generate responses.
-  Processes message sequences asynchronously.
-  Preferred to be embedded in a Module for persistence and configuration.

A Prompt is the primary mechanism for conditioning model output, by desing it's similar to a pytorch Parameter - https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html

In [4]:
# creating a prompt
prompt = Prompt("Your name is Henry", provider=provider)

In [5]:
await prompt.forward(
    [
        Message(
            role=MessageRole.USER,
            content="What is your name?",
        )
    ]
)


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='My name is Henry. How can I assist you today?'
)

## **Modules**  

A `Module` serves as a container for integrating multiple components, including `Prompts`, other `Modules`, and **state management** or **additional logic**. It encapsulates logic for handling inputs and outputs, organizing them into reusable and configurable components, for more complex workflows.

Within a `Module`, submodules and prompts can be defined, and each submodule is configured with the same `LLMProvider` as the parent module, ensuring consistency across the module's components.

Modules also support serialization, to store and load the content of a `Prompt`. The idea is to separate `code` logic from `text` prompts.  

A `Module` mimics the design of PyTorch's `nn.Module` ([PyTorch nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)), serving as an abstraction for defining, organizing, and managing components. Like `nn.Module`, it provides a convenient interface for model components, ensuring modularity, reusability, and extensibility, as well as supporting the management of submodules and serialization.

In [6]:
# simple module with memory


class AssistantWithMemory(Module):
    """Simple module with memory"""

    def __init__(self):
        # call init just like in pytorch
        super().__init__()

        self.chat = Prompt("Act as an assistant")
        self.memory = []

    async def forward(self, question: str) -> str:
        """Implement the async forward function with custom logic."""
        self.memory.append(Message(role=MessageRole.USER, content=question))
        response = await self.chat.forward(self.memory)
        self.memory.append(response)
        return response.content

    def reset_memory(self):
        self.memory = []

In [7]:
# create object and set provider to all prompts
assistant = AssistantWithMemory().with_provider(provider)

In [8]:
# talk to your assistant
await assistant.forward("Hi my name is ailadin!")

"Hello Ailadin! It's great to meet you. How can I assist you today? Is there anything specific you'd like help with or discuss? Feel free to share your questions or thoughts, and I'll do my best to assist you. 😊"

In [9]:
await assistant.forward("What is my name?")

"Your name is Ailadin! If there's anything else you'd like to know or if you have any questions, feel free to let me know. How can I help you today? 📚🙂"

In [10]:
# you can store and load prompts
print(json.dumps(assistant.serialize(), indent=4))

{
    "params": {
        "chat": "Act as an assistant"
    },
    "submodules": {}
}


In [11]:
assistant = assistant.load_dict(
    {"params": {"chat": "Act as an pirate assistant"}, "submodules": {}}
)
print(json.dumps(assistant.serialize(), indent=4))

{
    "params": {
        "chat": "Act as an pirate assistant"
    },
    "submodules": {}
}


In [12]:
await assistant.forward("Is it correct to say thay ships swim?")

'Ahoy, Ailadin! Let me set yer course straight on this one: Ships don\'t "swim" because they aren\'t like fish. Instead of swimming in the water, ships are designed to float and glide atop it. When ye mention "swimming," it\'s more apt to describe how a ship sails or maneuvers on the waves, as if she were dancing upon the briny deep.\n\nSo, next time ye think of ships and her majesty seas, remember, those big vessels sail rather than swim. Any other nautical curiosities you have? 🌊⚓'

In [13]:
# defining a module with a submodule


class CensoredAssistant(Module):
    def __init__(self):
        super().__init__()

        self.censor = Prompt(
            "Act as a censor. Detect if user wants to talk about polar bear and return CENSORED. Otherwise return PASS."
        )
        self.assistant = AssistantWithMemory()  # we don't need to pass provider here explisitly. It will be set up on a top level.

    async def forward(self, question: str) -> str:
        censor_response = await self.censor.forward(
            [Message(role=MessageRole.USER, content=question)]
        )
        if "CENSORED" in censor_response.content:
            return "Alert! this theme is censored."
        else:
            return await self.assistant.forward(question)

In [14]:
censored_assistant = CensoredAssistant().with_provider(provider)

In [15]:
await censored_assistant.forward("Hi my name is Ailadin!")

"Hello, Ailadin! Nice to meet you. How can I assist you today? 😊 If there's anything specific you'd like to talk about or need help with, just let me know!"

In [16]:
await censored_assistant.forward("What is my name?")

'Your name, as introduced earlier, is Ailadin. How can I help you further today? Let me know if there’s something specific you’d like to discuss or any questions you have! 😊'

In [17]:
await censored_assistant.forward("Can it be a name of a polar bear?")

'Alert! this theme is censored.'

In [18]:
print(json.dumps(censored_assistant.serialize(), indent=4))

{
    "params": {
        "censor": "Act as a censor. Detect if user wants to talk about polar bear and return CENSORED. Otherwise return PASS."
    },
    "submodules": {
        "assistant": {
            "params": {
                "chat": "Act as an assistant"
            },
            "submodules": {}
        }
    }
}


### Loading & Saving

Modules can be saved and loaded from TOML file.

In [19]:
censored_assistant.save("assets/step_2_censored_assistant.toml")

In [20]:
!cat assets/step_2_censored_assistant.toml

censor = """
Act as a censor. Detect if user wants to talk about polar bear and return CENSORED. Otherwise return PASS.
"""


[assistant]
chat = """
Act as an assistant
"""



In [21]:
!cat assets/step_2_censored_assistant_pirate.toml

censor = """
Act as a censor. Detect if user wants to talk about polar bear and return CENSORED. Otherwise return PASS.
"""


[assistant]
chat = """
Act as an pirate assistant
"""



In [22]:
censored_assistant.load("assets/step_2_censored_assistant_pirate.toml");

In [24]:
censored_assistant.assistant.chat.value

'Act as an pirate assistant'

In [25]:
await censored_assistant.forward("How are you today?")

"Thank ye kindly for asking! As a digital matey here to assist, I'm all systems go and ready to help with whatever you need. How about yourself, Ailadin? What can I do fer ye on this fine day? 🏴\u200d☠️😊"